In [7]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
conn = sqlite3.connect("ss.db")
cur  = conn.cursor()

import time
import os

import pandas as pd

qry = """SELECT bouncepoints_full.ID_Full, 
bouncepoints_full.Event, 
bouncepoints_full.Station, 
bouncepoints_full.Network, 
bouncepoints_full.Range, 
bouncepoints_full.MidLat, 
bouncepoints_full.MidLon, 
bouncepoints_full.EvtTime, 
bouncepoints_full.BAzimuth, 
downloads.Success
FROM bouncepoints_full
LEFT JOIN downloads 
ON bouncepoints_full.ID_Full=downloads.ID_Full
WHERE bouncepoints_full.MidLat > 30
and bouncepoints_full.MidLat < 75
and bouncepoints_full.MidLon < -60
and bouncepoints_full.MidLon > -135
and downloads.success == 'True' """

dfds = pd.read_sql_query(qry, conn, index_col=["ID_Full"])
dfds

,Event,Station,Network,Range,MidLat,MidLon,EvtTime,BAzimuth,Success
ID_Full,,,,,,,,,
C200601040832A.CZ.DPC,C200601040832A,DPC,CZ,89.488757,61.071432,-66.250616,2006-01-04T08:32:37.300000Z,316.859287,True
C200601040832A.CZ.DPC,C200601040832A,DPC,CZ,89.488757,61.071432,-66.250616,2006-01-04T08:32:37.300000Z,316.859287,True
C200601040832A.G.ECH,C200601040832A,ECH,G,86.562476,56.854058,-65.827059,2006-01-04T08:32:37.300000Z,310.063399,True
C200601040832A.G.ECH,C200601040832A,ECH,G,86.562476,56.854058,-65.827059,2006-01-04T08:32:37.300000Z,310.063399,True
C200601040832A.G.ECH,C200601040832A,ECH,G,86.562476,56.854058,-65.827059,2006-01-04T08:32:37.300000Z,310.063399,True
C200601040832A.G.MBO,C200601040832A,MBO,G,88.146495,30.232629,-61.582366,2006-01-04T08:32:37.300000Z,298.790014,True
C200601040832A.G.MBO,C200601040832A,MBO,G,88.146495,30.232629,-61.582366,2006-01-04T08:32:37.300000Z,298.790014,True
C200601040832A.G.SSB,C200601040832A,SSB,G,87.025584,54.713607,-64.079767,2006-01-04T08:32:37.300000Z,308.362185,True
C200601040832A.G.SSB,C200601040832A,SSB,G,87.025584,54.713607,-64.079767,2006-01-04T08:32:37.300000Z,308.362185,True


In [8]:
import obspy
from obspy.signal.filter import envelope
from scipy.signal import hilbert

from numpy import argmax, zeros

rbin = zeros(200)

snr ={}

for row in dfds.iterrows():
    bpindex = row[0]
    
    directory = 'data/mseed/%s/' % row[1].Event
    filename = directory + bpindex + '.mseed'
    st = obspy.read(filename).detrend().taper(0.05).filter('lowpass',freq=0.1,corners=4)
    
    tr = st[0]
    
    noise   = tr.copy().trim(tr.stats.starttime, tr.stats.starttime+100)
    signal  = tr.copy().trim(tr.stats.starttime+100, tr.stats.starttime+200)
    
    if len(noise.data) == 0 or len(signal.data) == 0:
        snr[bpindex] = -1.0
        continue
    
    snr[bpindex]    = max(abs(envelope(signal.data))) / max(abs(envelope(noise.data)))
    

    

In [9]:
snr.items()
dfout = pd.DataFrame(snr, index=["SNR"]).T
dfout

,SNR
B201305241456A.C.GO01,2.157818
B201305241456A.C.GO02,1.099111
B201305241456A.CU.ANWB,0.920397
B201305241456A.CU.BBGH,0.878139
B201305241456A.CU.BCIP,1.247134
B201305241456A.CU.GRGR,0.694184
B201305241456A.CU.GRTK,1.240578
B201305241456A.CU.GTBY,1.145152
B201305241456A.CU.MTDJ,1.383752
B201305241456A.CU.SDDR,1.033844


In [10]:
#dfout.to_csv('snr.csv')
dfout.to_sql('snr',conn, if_exists="replace", index_label="ID_Full")

In [11]:
conn.close()